In [ ]:
# default_exp uniformis

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

# Uniform IS

## Basic Settings

In [ ]:
# imports
from bfh_mt_hs2020_sec_data.core import * 
from pathlib import Path
from typing import List, Tuple, Union, Set
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col

import pandas as pd

import shutil          # provides high level file operations
import time            # used to measure execution time
import os
import sys

In [ ]:
all_pivot_selected_folder  = "D:/data_mt/03_1_selected_pivot_cols_parquet" 
all_pivoted_folder         = "D:/data_mt/03_2_split_pivot_parquet"         
all_processed_folder       = "D:/data_mt/04_unified/"

col_list =    ["stmt","cik","ticker", "adsh","period","filed", "form","tag","value","report", "line", "fp", "uom"]
pivot_group = ["cik","ticker","adsh","form","period","filed","fp", "qtrs"]

In [ ]:
# init Spark
spark = get_spark_session() # Session anlegen
spark # display the moste important information of the session

## 00_Raw_data

In [ ]:
# loading the complete unpivoted dataset - if it is needed for debbuging
df_all_selected = spark.read.parquet(all_pivot_selected_folder).cache()

In [ ]:
# it happens sometimes, that the data could not be associated with a right sheet (bs, is, cf, ..). in this cases, the data can appea under "UN"
# so if expected information cannot be found in the appropriate statement, we have to look in the un statement
un_pivot_value = load_data(all_pivoted_folder, spark, "UN", "value")
un_pivot_value = un_pivot_value.where("((form == '10-K' and qtrs == '4') or (form == '10-Q' and qtrs == '1'))")
un_pivot_pd = un_pivot_value.toPandas()

In [ ]:
un_pivot_pd.shape

(2220, 1775)

In [ ]:
def prepare_un_values(df_to_merge_into, attr_list):
    # add possible columns from un set to  cf data with prefix cpy_
    attributes = pivot_group[:] # create copy
    attributes.extend(attr_list)

    un_prepared = un_pivot_pd[attributes].copy()
    un_prepared.rename(columns=lambda x: x  if x in pivot_group else ("cpy_" + x), inplace=True)

    return pd.merge(df_to_merge_into, un_prepared, how='left', on=pivot_group)

## 01_IncomeStatement

Gross Margin
- Net Sales
- Cost of Sales
- Gross Margin -> NetSales - CostOfSales

Operating Expenses
- R&D
- Selling, general and admin
- Total op expenses = R&D + Selling, general and admin

- Operating Income = Gross Margin - Total op expenses      -> OperatingIncomeLoss
- other income
- Income before provision for income taxes = operating income + other income

- Provision for income taxes
- Net income = Income before taxes -taxes                   -> NetIncomeLoss -> also available in CF(!)

Earning per share
- Basic
- Diluted

Shares used in computing earnings per share:
- basic
- diluted


In [ ]:
is_pivot_value = load_data(all_pivoted_folder, spark, "IS", "value")
spark_shape(is_pivot_value)

(116298, 1928)

In [ ]:
is_pivot_pd = is_pivot_value.toPandas()

In [ ]:
is_pivot_pd_copy = is_pivot_pd.copy()

In [ ]:
is_pivot_pd_copy.shape

(116298, 1929)

### Merge Data from "unknown" statement

In [ ]:
# merge relevant columns from the UN dataset
is_pivot_pd_copy = prepare_un_values(is_pivot_pd_copy, [    
    'NetIncomeLoss',
    'NetIncomeLossAvailableToCommonStockholdersBasic',
    'NetIncomeLossAllocatedToLimitedPartners',
    'ProfitLoss',
    
    'Revenues',
    'SalesRevenueNet',
    'SalesRevenueGoodsNet',
    'SalesRevenueServicesNet',
    'RevenueFromContractWithCustomerExcludingAssessedTax', 
    'RevenueFromContractWithCustomerIncludingAssessedTax', 
    'CostOfGoodsAndServicesSold',
    'CostOfGoodsSold',
    'CostOfRevenue',
    'CostOfServices',
    'CostsAndExpenses',
    'OperatingIncomeLoss',
    'IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments',
    'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest',
    
    'GrossProfit',
    'OperatingExpenses',
])
is_pivot_pd_copy.shape

(116298, 1949)

### NetIncome

In [ ]:
print_null_count(is_pivot_pd_copy, [ 'NetIncomeLoss', 'NetIncomeLossAvailableToCommonStockholdersBasic', 'ProfitLoss'])

NetIncomeLoss   15356
NetIncomeLossAvailableToCommonStockholdersBasic   92287
ProfitLoss   76841


In [ ]:
is_pivot_pd_copy['c_NetIncomeLoss'] = None

copy_if_not_empty(is_pivot_pd_copy, 'cpy_NetIncomeLoss', 'NetIncomeLoss')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_NetIncomeLossAvailableToCommonStockholdersBasic', 'NetIncomeLossAvailableToCommonStockholdersBasic')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_NetIncomeLossAllocatedToLimitedPartners', 'NetIncomeLossAllocatedToLimitedPartners')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_ProfitLoss', 'ProfitLoss')


copy_if_not_empty(is_pivot_pd_copy, 'NetIncomeLoss', 'c_NetIncomeLoss')
copy_if_not_empty(is_pivot_pd_copy, 'NetIncomeLossAvailableToCommonStockholdersBasic', 'c_NetIncomeLoss')
copy_if_not_empty(is_pivot_pd_copy, 'NetIncomeLossAllocatedToLimitedPartners', 'c_NetIncomeLoss')
copy_if_not_empty(is_pivot_pd_copy, 'ProfitLoss', 'c_NetIncomeLoss')

In [ ]:
print_null_count(is_pivot_pd_copy, [ 'c_NetIncomeLoss', 'NetIncomeLoss', 'ProfitLoss'])

c_NetIncomeLoss   879
NetIncomeLoss   15306
ProfitLoss   76791


### NetSales / Revenues

In [ ]:
print_null_count(is_pivot_pd_copy, [ 
    'Revenues',
    'SalesRevenueNet',
    'SalesRevenueGoodsNet',
    'SalesRevenueServicesNet',
    'SalesRevenueGoodsGross',
    'ContractsRevenue',
    'RevenueOilAndGasServices',
    'HealthCareOrganizationPatientServiceRevenue',
    'HealthCareOrganizationRevenue',
    'RevenueFromContractWithCustomerExcludingAssessedTax', # Sales
    'RevenueFromContractWithCustomerIncludingAssessedTax', # Sales
    'RevenueMineralSales',
    'SalesRevenueEnergyServices',
    'RealEstateRevenueNet',
])

Revenues   73226
SalesRevenueNet   86102
SalesRevenueGoodsNet   98319
SalesRevenueServicesNet   105676
SalesRevenueGoodsGross   115801
ContractsRevenue   114648
RevenueOilAndGasServices   115813
HealthCareOrganizationPatientServiceRevenue   115999
HealthCareOrganizationRevenue   115862
RevenueFromContractWithCustomerExcludingAssessedTax   106217
RevenueFromContractWithCustomerIncludingAssessedTax   113095
RevenueMineralSales   116030
SalesRevenueEnergyServices   116157
RealEstateRevenueNet   114136


In [ ]:
is_pivot_pd_copy['c_Revenues'] = None

copy_if_not_empty(is_pivot_pd_copy, 'cpy_Revenues', 'Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_SalesRevenueNet', 'SalesRevenueNet')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_SalesRevenueGoodsNet', 'SalesRevenueGoodsNet')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_SalesRevenueServicesNet', 'SalesRevenueServicesNet')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_RevenueFromContractWithCustomerExcludingAssessedTax', 'RevenueFromContractWithCustomerExcludingAssessedTax')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_RevenueFromContractWithCustomerIncludingAssessedTax', 'RevenueFromContractWithCustomerIncludingAssessedTax')
#copy_if_not_empty(is_pivot_pd_copy, 'cpy_RevenuesExcludingInterestAndDividends', 'RevenuesExcludingInterestAndDividends')

copy_if_not_empty(is_pivot_pd_copy, 'Revenues', 'c_Revenues')

sum_into_empty_target(is_pivot_pd_copy,  
                      'SalesRevenueGoodsNet',
                      'SalesRevenueServicesNet',
                      'SalesRevenueNet')

copy_if_not_empty(is_pivot_pd_copy, 'SalesRevenueNet', 'c_Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'SalesRevenueGoodsNet', 'c_Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'SalesRevenueServicesNet', 'c_Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'RevenueFromContractWithCustomerExcludingAssessedTax', 'c_Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'RevenueFromContractWithCustomerIncludingAssessedTax', 'c_Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'RevenuesExcludingInterestAndDividends', 'c_Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'RegulatedAndUnregulatedOperatingRevenue', 'c_Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'HealthCareOrganizationPatientServiceRevenue', 'c_Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'SalesRevenueGoodsGross', 'c_Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'ContractsRevenue', 'c_Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'RevenueOilAndGasServices', 'c_Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'HealthCareOrganizationRevenue', 'c_Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'RevenueMineralSales', 'c_Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'SalesRevenueEnergyServices', 'c_Revenues')
copy_if_not_empty(is_pivot_pd_copy, 'RealEstateRevenueNet', 'c_Revenues')

In [ ]:
# some companies provide NonInterestIncome and InterestAndDividendIncomeOperating instead of a Revenue
sum_into_empty_target(is_pivot_pd_copy,  
                      'InterestAndDividendIncomeOperating',
                      'NoninterestIncome',
                      'c_Revenues')

sum_into_empty_target(is_pivot_pd_copy,  
                      'InterestIncomeExpenseNet',
                      'NoninterestIncome',
                      'c_Revenues')

In [ ]:
# other revenue types, ordered frequency of usage
other_revenue_types = \
['OtherSalesRevenueNet', 'OperatingLeasesIncomeStatementLeaseRevenue', 'LicensesRevenue', 'RevenueFromRelatedParties',
'BrokerageCommissionsRevenue', 'RoyaltyRevenue', 'OilAndGasSalesRevenue', 'OilAndGasRevenue', 'OtherRealEstateRevenue',
'TechnologyServicesRevenue', 'ManagementFeesRevenue', 'ReimbursementRevenue', 'OperatingLeasesIncomeStatementMinimumLeaseRevenue',
'FoodAndBeverageRevenue', 'MaintenanceRevenue', 'LicenseAndServicesRevenue', 'FranchiseRevenue', 'SubscriptionRevenue', 'FinancialServicesRevenue',
'RevenueFromGrants', 'GasGatheringTransportationMarketingAndProcessingRevenue', 'OccupancyRevenue', 'NaturalGasProductionRevenue',
'SalesRevenueServicesGross', 'InvestmentBankingRevenue', 'AdvertisingRevenue', 'RevenueOtherFinancialServices',
'OilAndCondensateRevenue', 'RevenueFromLeasedAndOwnedHotels', 'RevenuesNetOfInterestExpense', 'RegulatedAndUnregulatedOperatingRevenue',
'UnregulatedOperatingRevenue', 'ElectricUtilityRevenue', 'CargoAndFreightRevenue', 'OtherHotelOperatingRevenue',
'CasinoRevenue', 'RefiningAndMarketingRevenue', 'PrincipalTransactionsRevenue', 'InterestRevenueExpenseNet',
'HomeBuildingRevenue', 'OtherRevenueExpenseFromRealEstateOperations', 'GasDomesticRegulatedRevenue', 'LicenseAndMaintenanceRevenue',
'RegulatedOperatingRevenue', 'AdmissionsRevenue','PassengerRevenue',]

for revenue in other_revenue_types: 
    copy_if_not_empty(is_pivot_pd_copy, revenue, 'c_Revenues')

In [ ]:
set_to_zero_if_null(is_pivot_pd_copy, 'c_Revenues')

In [ ]:
print_null_count(is_pivot_pd_copy, [ 'c_Revenues'])

c_Revenues   0


### CostOfSales

In [ ]:
print_null_count(is_pivot_pd_copy, [ 
    'CostOfGoodsAndServicesSold',
    'CostOfGoodsSold',
    'CostOfRevenue',
    'CostOfServices',
])

CostOfGoodsAndServicesSold   93233
CostOfGoodsSold   92487
CostOfRevenue   97675
CostOfServices   106987


In [ ]:
is_pivot_pd_copy['c_CostOfRevenue'] = None

copy_if_not_empty(is_pivot_pd_copy, 'cpy_CostOfGoodsAndServicesSold', 'CostOfGoodsAndServicesSold')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_CostOfGoodsSold', 'CostOfGoodsSold')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_CostOfRevenue', 'CostOfRevenue')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_CostOfServices', 'CostOfServices')

copy_if_not_empty(is_pivot_pd_copy, 'CostOfRevenue', 'c_CostOfRevenue')
copy_if_not_empty(is_pivot_pd_copy, 'CostOfGoodsAndServicesSold', 'c_CostOfRevenue')

sum_into_empty_target(is_pivot_pd_copy,  
                      'CostOfGoodsSold',
                      'CostOfServices',
                      'c_CostOfRevenue')

copy_if_not_empty(is_pivot_pd_copy, 'CostOfGoodsSold', 'c_CostOfRevenue')
copy_if_not_empty(is_pivot_pd_copy, 'CostOfServices', 'c_CostOfRevenue')

In [ ]:
print_null_count(is_pivot_pd_copy, ['c_CostOfRevenue'])

c_CostOfRevenue   52766


### OperatingIncomeLoss

In [ ]:
print_null_count(is_pivot_pd_copy, ['OperatingIncomeLoss',
                                   'IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments',
                                   'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest'])

OperatingIncomeLoss   29317
IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments   60211
IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest   70831


In [ ]:
is_pivot_pd_copy['c_OperatingIncomeLoss'] = None

copy_if_not_empty(is_pivot_pd_copy, 'cpy_OperatingIncomeLoss', 'OperatingIncomeLoss')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments', 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest', 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest')

copy_if_not_empty(is_pivot_pd_copy, 'OperatingIncomeLoss', 'c_OperatingIncomeLoss')
copy_if_not_empty(is_pivot_pd_copy, 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments', 'c_OperatingIncomeLoss')
copy_if_not_empty(is_pivot_pd_copy, 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest', 'c_OperatingIncomeLoss')

In [ ]:
print_null_count(is_pivot_pd_copy, ['c_OperatingIncomeLoss'])

c_OperatingIncomeLoss   4421


### Other

In [ ]:
copy_if_not_empty(is_pivot_pd_copy, 'cpy_CostsAndExpenses', 'CostsAndExpenses')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_GrossProfit', 'GrossProfit')
copy_if_not_empty(is_pivot_pd_copy, 'cpy_OperatingExpenses', 'OperatingExpenses')

#### Save

In [ ]:
is_pivot_pd_copy[["cik","ticker", "adsh","period","filed", "form", "qtrs","fp",
                    'c_Revenues',
                    'c_CostOfRevenue',
                    'GrossProfit',
                    'OperatingExpenses',
                    'c_OperatingIncomeLoss',
                    'CostsAndExpenses',
                    'c_NetIncomeLoss'
                 ]] \
                  .to_csv(all_processed_folder + "04_is_unified.csv", index=False)

In [ ]:
spark.stop()

## XX_Trials

In [ ]:
list_cols = [x for x in is_pivot_pd_copy.columns.values if ('OperatingExpenses' in x)]
list_cols

['OperatingExpenses']

In [ ]:
pd.set_option('display.max_rows',50)
list_cols = [x for x in is_pivot_pd_copy.columns.values if ('Revenue' in x)]
#[x for x in is_pivot_pd_copy.columns.values if ('Revenue' in x) and ('Customer' in x)]

selected_cols = set(list_cols) - set([
    'Revenues_hj',
    'CostOfRevenue_hj',
    'costOfRevenue',
    'Revenues',
    'SalesRevenueNet',
    'SalesRevenueGoodsNet',
    'SalesRevenueServicesNet',
    'SalesRevenueGoodsGross',
    'ContractsRevenue',
    'RevenueOilAndGasServices',
    'HealthCareOrganizationPatientServiceRevenue',
    'HealthCareOrganizationRevenue',
    'RevenueFromContractWithCustomerExcludingAssessedTax', # Sales
    'RevenueFromContractWithCustomerIncludingAssessedTax', # Sales
    'RevenueMineralSales',
    'SalesRevenueEnergyServices',
    'RealEstateRevenueNet',])

is_pivot_pd_copy[selected_cols].isnull().sum(axis=0).sort_values()[:50]

CostOfRevenue                                               97275
OtherSalesRevenueNet                                       110435
OperatingLeasesIncomeStatementLeaseRevenue                 112498
LicensesRevenue                                            113529
RevenueFromRelatedParties                                  114129
BrokerageCommissionsRevenue                                114288
RoyaltyRevenue                                             114314
OilAndGasSalesRevenue                                      114320
OilAndGasRevenue                                           114473
OtherCostOfOperatingRevenue                                114586
OtherRealEstateRevenue                                     114673
TechnologyServicesRevenue                                  114754
ManagementFeesRevenue                                      114849
ReimbursementRevenue                                       114882
OperatingLeasesIncomeStatementMinimumLeaseRevenue          114929
FoodAndBev